# Einführung in die Materialmodellierung - Übung 03 Rheologie 2

## Aufgabe 3.1
**Leiten Sie die Differentialgleichung des dargestellten Modells her**

![model](figures/model2_solution.png)

**Spannungsbeziehungen**
\begin{equation}
\sigma = \sigma_1 + \sigma_2 + \sigma_3 
\end{equation}

**Dehnungsbeziehungen**
\begin{equation}
\varepsilon = \varepsilon_1 + \varepsilon_2 
\end{equation}

**Dehnungsrate**
\begin{equation}
\dot{\varepsilon} = \dot{\varepsilon}_1 + \dot{\varepsilon}_2
\end{equation}

**Elementbeziehungen bzw. Materialgesetze**
\begin{equation}
\sigma = E\varepsilon_2 
\end{equation}

\begin{equation}
\sigma_3 = E_1 \varepsilon_1 
\end{equation}

\begin{equation}
\sigma_1 = \eta \dot{\varepsilon}_1
\end{equation}

St.-Venant Element
\begin{equation}
f_2 = |\sigma_2| - \sigma_y 
\end{equation}

\begin{equation}
\dot{\varepsilon}_1 = \dot{\gamma}_2 \text{ sign}(\sigma_2)
\end{equation}

\begin{equation}
\dot{\gamma}_2 \geq 0, \qquad f_2 \leq 0, \qquad \dot{\gamma}_2f_2=0
\end{equation}

---

**Differentialgleichung**

Sobald St.-Venant Elemente im Modell vorkommen, müssen Fallunterscheidungen vorgenommen werden für $\sigma < \sigma_f$ (1), $\sigma\geq\sigma_f$ (2). Sind mehrere St.-Venant Modelle im Modell, müssen zusätzliche Fälle betrachtet werden.

Fall (1) $\sigma < \sigma_f$:
\begin{equation}
\dot{\gamma}_2 = 0 \rightarrow \dot{\varepsilon}_1 = 0 \rightarrow \varepsilon_1=0 \text{ (Initialkond.)} 
\end{equation}

\begin{equation}
\sigma = E\varepsilon_2 = E(\varepsilon - \varepsilon_1) 
\end{equation}

\begin{equation}
\dot{\sigma} = E (\dot{\varepsilon} - \dot{\varepsilon}_1) = E \dot{\varepsilon}
\end{equation}

Fließgrenze
\begin{equation}
f_2=0 \rightarrow \sigma_2 = \sigma_y  \qquad \varepsilon_1=0\rightarrow \sigma=\sigma_2+\underbrace{\sigma_1+\sigma_3}_{=0} = \sigma_y
\end{equation}

\begin{equation}
\varepsilon = \frac{\sigma}{E} = \frac{\sigma_y}{E} =: \varepsilon_y
\end{equation}

Fall (2) $\sigma \geq \sigma_f$:
\begin{equation}
\dot{\gamma}_2>0\rightarrow \dot{\varepsilon}_1 \neq 0 \rightarrow  f_2=0 \rightarrow \sigma_2 = \sigma_y
\end{equation}

\begin{equation}
\sigma = \sigma_1+\sigma_2+\sigma_3 
\end{equation}

\begin{equation}
\sigma = \eta \dot{\varepsilon}_1 + \sigma_y + E_1 \varepsilon_1
\end{equation}

\begin{align}
	\sigma &= \eta \left(\dot{\varepsilon} - \dot{\varepsilon}_2\right) + \sigma_y + E(\varepsilon -\varepsilon_2)\\
	&= \eta \left(\dot{\varepsilon} - \frac{\dot{\sigma}}{E}\right) + \sigma_y + E_1 \left(\varepsilon - \frac{\sigma}{E}\right)
\end{align}

Als letzten Schritt werden noch alle $\sigma$ bzw. $\varepsilon$ Terme auf eine Seite geordnet und erhält somit
\begin{equation}
	\sigma\left(1+\frac{E_1}{E}\right)+\frac{\eta}{E}\dot{\sigma} - \sigma_y = \eta\dot{\varepsilon}+E_1\varepsilon
\end{equation}

**Numerische Lösung**

Substitution der Zeitableitungen mit den Differenzenquotienten ergibt
\begin{equation}
\frac{\sigma_{n+1}-\sigma_n}{\Delta t} = E \frac{\varepsilon_{n+1}-\varepsilon_n}{\Delta t} \quad \text{für } \sigma<\sigma_y 
\end{equation}

\begin{equation}
	\sigma_{n+1}\left(1+\frac{E_1}{E}\right)+\frac{\eta}{E}\frac{\sigma_{n+1} - \sigma_n}{\Delta t} - \sigma_y = \eta\frac{\varepsilon_{n+1}-\varepsilon_n}{\Delta t}+E_1\varepsilon_{n+1} \quad \text{für } \sigma\geq \sigma_y
\end{equation}

Nun kann nach $\sigma$ bzw. $\varepsilon$ aufgelöst werden. Beispielhaft für $\varepsilon$:
\begin{equation}
\varepsilon_{n+1} \frac{E\varepsilon_n + \sigma-\sigma_n}{E} \qquad \text{für } \sigma < \sigma_y
\end{equation}

\begin{equation}
\varepsilon_{n+1} = \frac{E\Delta t (\sigma_{n+1} - \sigma_y)+E\eta\varepsilon_n+E_1\Delta t \sigma_{n+1} + \eta(\sigma_{n+1} - \sigma_n)}{E(E_1\Delta t+\eta)} \qquad \text{für } \sigma \geq \sigma_y
\end{equation}

mit Initial- und Endkondition
\begin{equation}
 \varepsilon(0)=\frac{\sigma}{E}\qquad \text{und} \qquad \varepsilon(\infty) = \frac{\tilde{\sigma}}{E_1} + \frac{\tilde{\sigma}}{E} -\frac{\sigma_y}{E_1}
\end{equation}

---
---

In [110]:
using Plots;
plotly()
t = collect(0:1:100);

In [111]:
function constant(max::Float64, t::Array)
    load = max * ones(length(t))
    return load
end

function constant_cycle(max::Float64, t::Array)
    T = length(t)
    load = max * ones(T)
    load[(T÷4+1):(T÷2)] .= 0
    load[(T*3÷4+1):T] .= 0
    return load
end

function sinus(max::Float64, t::Array)
    load = max * sin.(8*pi/100*t)
    return load
end

function linear(max::Float64, t::Array)
    T = length(t)
    load = collect(0:max/(T-1):max)
    return load
end

function linear_constant(max::Float64, t::Array)
    T = length(t)
    load = copy(t)
    load = map(x -> 2*max/(T) *x, load)
    load[length(t)÷2+1:end] .= max
    return load
end

linear_constant (generic function with 1 method)

In [112]:
plot(constant(0.1, t), ylabel="ε oder σ", xlabel="Zeit t", label="konstant", line=5)
plot!(constant_cycle(0.1, t), line=5, label="zyklisch konstant")
plot!(linear(0.1, t), line=5, label="linear")
plot!(linear_constant(0.1,t), line=5, label="linear konstant")

<!DOCTYPE html>
 
 
 Plots.jl

In [113]:
function viscoplasticity_ε(E::Float64, E₁::Float64, η::Float64, σ_y::Float64, σ::Array, t::Array)
    ε = zeros(length(t))
    Δσ = zeros(length(t))
    ε[1] = σ[1]/E
    for i in 1:length(t)-1
        if σ[i+1] < σ_y
            ε[i+1] = (E*ε[i]+σ[i+1]-σ[i])/(E)
        else
            Δt = t[i+1] - t[i]
            Δσ[i+1] = σ[i+1] - σ[i]
            ε[i+1] = (E*Δt*(σ[i+1]-σ_y)+E*η*ε[i]+E₁*Δt*σ[i+1]+η*(σ[i+1]-σ[i]))/(E*(E₁*Δt+η))
        end
    end
    return ε, Δσ
end


viscoplasticity_ε (generic function with 1 method)

In [114]:
E = 100.
E₁ = 20.
η = 100.
σ_y = 2.
ε_viscoplasticity, Δε_viscoplasticity = viscoplasticity_ε(E, E₁, η, σ_y, linear_constant(3., t), t);
plot(t,ε_viscoplasticity, ylabel="ε", xlabel="Zeit t", label="Viscoplasticity", line=5)

<!DOCTYPE html>
 
 
 Plots.jl

Endbedingung
$$\varepsilon(\infty)=\frac{\tilde{\sigma}}{E_1} +\frac{\tilde{\sigma}}{E} - \frac{\sigma_y}{E}$$

In [115]:
3/E₁ + 3/E - σ_y/E₁

0.07999999999999999

## Zusätzliche Informationen

### Weitere viskoplastische Modelle und deren DGLs sind unter [diesem Wikipedia Artikel zu finden](https://en.wikipedia.org/wiki/Viscoplasticity)